# Setup

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from graphviz import Digraph

In [ ]:
print(2+2)

# Micrograd

Github library `karpathy/micrograd`

- Autograd engine
- Implements back-propagation
- Efficiently evaluate the gradient of a loss function wrt the weights of a neural network
- Tune the weights of a neural network efficiently
- Builds the graph automatically
    - Forward and backward pass
- Backward pass: chain rule to calculate the derivatives
    - How values effect others through the gradient

Neural networks

- Just mathematical expression
- Data + weights $\rightarrow$ Outputs
- Backprop is not specific to neural networks

Micrograd uses scalars, but torch will use tensors.

- Math does not change on tensors
- Tensors are arrays of tensors
- Parallelized
- Everything else is efficiency


# Derivatives

In [ ]:
def f(x):
    return 3 * x ** 2 - 4 * x + 5

In [ ]:
f(3.0)

xs = np.arange(-5, 5.25, 0.25)
ys = f(xs)

plt.plot(xs, ys);

Derivatives without derivatives.

- NN doesn't do this symbolically.
- What is the sensitivity of a function at a certain value of $x$?

Numerical approximation of the slope at $x$:

In [ ]:
h = 0.001
x = 3.0

(f(x + h) - f(x)) / h

- Rise: $(f(x + h) - f(x))$
- Run: $h$

Negative slopes at $x = -3$:

In [ ]:
h = 0.001
x = -3.0

(f(x + h) - f(x)) / h

Zero slope at $x = 2/3$

In [ ]:
h = 0.001
x = 2.0 / 3.0

(f(x + h) - f(x)) / h

# Function with three scalar inputs

In [ ]:
a = 2.0
b = -3.0
c = 10.0
d = a * b + c
print(d)

Derivative of $d$ as a function of $a$, $b$, and $c$

In [ ]:
h = 0.001

# inputs
a = 2.0
b = -3.0
c = 10

d1 = a * b + c
a += h
d2 = a * b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1) / h)

d1 = a * b + c
b += h
d2 = a * b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1) / h)

d1 = a * b + c
c += h
d2 = a * b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1) / h)

# Data structures to hold values

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward

        return out

    # Alternate mul for 2 * a
    def __rmul__(self, other): # other * self
        return self * other

    def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - t ** 2) * out.grad

        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


a = Value(2.0, label = 'a')
b = Value(-3.0, label = 'b')
print(a + b)

c = Value(10.0, label = 'c')
print(a * b)

# Previous, children, operation

We now know how each value was created

?? Should be -6 and 10, not 10 and -6 ??

In [ ]:
e = a * b
e.label = 'e'
d = e + c
d.label = 'd'
f = Value(-2.0, label = 'f')
L = d * f
L.label = 'L'
print(L)

# Visualization of expression graphs

In [ ]:
def trace(root):
    nodes, edges = set(), set()

    # Build a set of nodes and edges
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # L to R
    nodes, edges = trace(root)

    for n in nodes:
        uid = str(id(n))

        # for any value in the graph create a rectangular record
        dot.node(name = uid,
                 label = "{ %s | data %.4f | grad %0.4f}" % (n.label, n.data, n.grad),
                 shape='record')
        if n._op:
            # if the value is a result of an operation, create an op node
            dot.node(name = uid + n._op, label = n._op)

            # and connect this node to it
            dot.edge(uid + n._op, uid)
    for n1, n2, in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

draw_dot(L)

Output of the forward pass is

In [ ]:
print(L.data)

## Back-propagation

Start at $L$ and calculate the gradient in reverse.

- Derivative of each node wrt $L$
- Data is fixed, so we don't calculate the derivatives for data
- Recursively multiply the derivatives along the path

$$
\frac{dL}{dL} = 1
$$

Chain rule

$$
\frac{dz}{dx} = \frac{dz}{dy} \cdot \frac{dy}{dx}
$$

Gradient gives you the power to influence the output.


## Back-propagation through a neuron

tanh activation function

- Inputs are squashed to the range -1 and 1
- Useful in the range -2 to 2


In [ ]:
plt.figure()
plt.plot(np.arange(-5, 5, 0.2), np.tanh(np.arange(-5, 5, 0.2)))
plt.grid()

Weights

In [ ]:
# inputs x1 and x2
x1 = Value(2.0, label = 'x1')
x2 = Value(0.0, label = 'x2')

# weights w1 and w2
w1 = Value(-3.0, label = 'w1')
w2 = Value(1.0, label = 'w2')

# b is the bias of the neuron
b = Value(6.8813735870195432, label = 'b')

x1w1 = x1 * w1; x1w1.label = 'x1 * w1'
x2w2 = x2 * w2; x2w2.label = 'x2 * w2'

x1w1x2w1 = x1w1 + x2w2; x1w1x2w1.label = '(x1 * w1) + (x2 * w2)'

# n is the cell body activation without the activation function
n = x1w1x2w1 + b; n.label = 'n'

draw_dot(n)

# Implement tanh above
o = n.tanh(); o.label = 'o'
draw_dot(o)

Derivatives on the weights is the important part

Back-propagation manually from $o$:

In [ ]:
# o.grad = 1.0
# n.grad = 0.5
# x1w1x2w1.grad = 0.5
# b.grad = 0.5
# x1w1.grad = 0.5
# x2w2.grad = 0.5
# x2.grad = w2.data * x2w2.grad
# w2.grad = x2.data * x2w2.grad
# x1.grad = w1.data * x1w1.grad
# w1.grad = x1.data * x1w1.grad

# # o = tanh(n)
# # ? do / dn = 1 - tanh(n)**2

# 1 - o.data ** 2

# draw_dot(o)

# Backward pass automatically

Implement `backward` above

In [ ]:
o.grad = 1.0
o._backward()
n._backward()
b._backward()
x1w1x2w1._backward()
x1w1._backward()
x2w2._backward()
draw_dot(o)

We don't want to have to set `o.grad` manually. Use topological sort of the DAG left to right.

In [ ]:
o.backward()
draw_dot(o)

Bug: Gradient is incorrect (should be 2). Gradient is overwritten because self and other point to the same memory location.

- Need to accumulate the derivatives `+=`.

In [ ]:
a = Value(3.0, label = 'a')
b = a + a; b.label = 'b'
b.backward()
draw_dot(b)

# With `micrograd`

In [ ]:
from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP

y = [1.0, -1.0, -1.0, 1.0]
X = [[2.0, 3.0, -1.0],
     [3.0, -1.0, 0.5],
     [0.5, 1.0, 1.0],
     [1.0, 1.0, -1.0]]

print(ys)
print(xs)

## Initialize a model

In [ ]:
model = MLP(2, [16, 16, 1]) # 2-layer neural network
print(model)
print("number of parameters", len(model.parameters()))

## Define a loss function

In [ ]:
def loss(batch_size=None):

    # inline DataLoader :)
    if batch_size is None:
        Xb, yb = X, y
    else:
        ri = np.random.permutation(X.shape[0])[:batch_size]
        Xb, yb = X[ri], y[ri]
    inputs = [list(map(Value, xrow)) for xrow in Xb]

    # forward the model to get scores
    scores = list(map(model, inputs))

    # svm "max-margin" loss
    losses = [(1 + -yi*scorei).relu() for yi, scorei in zip(yb, scores)]
    data_loss = sum(losses) * (1.0 / len(losses))
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    total_loss = data_loss + reg_loss

    # also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(yb, scores)]
    return total_loss, sum(accuracy) / len(accuracy)

total_loss, acc = loss()
print(total_loss, acc)


## Optimization

In [ ]:
for k in range(100):

    # forward
    total_loss, acc = loss()

    # backward
    model.zero_grad()
    total_loss.backward()

    # update (sgd)
    learning_rate = 1.0 - 0.9*k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad

    if k % 1 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")

## Decision boundary

In [ ]:
h = 0.25
X = np.array(X)  # Convert list to NumPy array
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Xmesh = np.c_[xx.ravel(), yy.ravel()]
inputs = [list(map(Value, xrow)) for xrow in Xmesh]
scores = list(map(model, inputs))
Z = np.array([s.data > 0 for s in scores])
Z = Z.reshape(xx.shape)

fig = plt.figure()
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())

# With torch

In [ ]:
import torch

x1 = torch.Tensor([2.0]).double()                ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()                ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()               ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()                ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double()  ; b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

In [ ]:
class Neuron:

  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out

  def parameters(self):
    return self.w + [self.b]

class Layer:

  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:

  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [ ]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)